In [1]:
import tensorflow as tf

2022-11-26 20:47:28.334357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 20:47:28.564852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/robin/.guix-home/profile/lib
2022-11-26 20:47:28.564888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-26 20:47:28.604271: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-26 20:4

In [2]:
model = tf.keras.models.load_model('saved_model/tf_model')


2022-11-26 20:47:32.038455: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/robin/.guix-home/profile/lib
2022-11-26 20:47:32.038476: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-26 20:47:32.038494: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ada): /proc/driver/nvidia/version does not exist
2022-11-26 20:47:32.038847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


```glsl
f16 inputs[113];

f16 hidden_layer_values_a[30];
f16 hidden_layer_values_b[30];

f16 layer0_weights[113][30];
f16 layer0_bias[30];
hidden_layer_values_a = relu(matmul(layer0_weights, inputs) + layer0_bias)

f16 layer1_weights[30][30];
f16 layer1_bias[30];
hidden_layer_values_b = relu(matmul(layer1_weights, hidden_layer_values_a) + layer1_bias)

f16 layer2_weights[30][30];
f16 layer2_bias[30];
hidden_layer_values_a = relu(matmul(layer1_weights, hidden_layer_values_b) + layer2_bias)

....


f16 layer6_weights[30];
f16 layer6_bias;
f16 output = matmul(layer6_weights, hidden_layer_values_b) + layer6_bias
```

In [21]:
from textwrap import dedent, indent
from pathlib import Path
# def make_unique(name, *, **, dummy=[0]):
#     n = name + f"_{dummy}"
#     dummy[0] += 1
#     return n

out = "shader.comp"
shader_code = ""

def print_out(output, wrap=100):
    for line in output.splitlines():
        if len(line) > wrap:
            print(line[:wrap] + "...")
        else:
            print(line)

def dump(value):
    global shader_code
    shader_code += value + "\n"
    
def array_to_init_string(values):
    return "{" + ", ".join(map(str, values)) + "}"

def array2d_to_init_string(values):
    return "{" + ", ".join(map(array_to_init_string, values)) + "}"
    

    
weights = []
biases = []
activation_func = []
for layer in model.layers:
    activation_function = layer.activation.__name__ 
    assert activation_function in ["relu", "linear"]
    activation_func.append(activation_function)
    
    for w in layer.weights:
        if "kernel" in w.name:
            weights.append(w.numpy())
        elif "bias" in w.name:
            biases.append(w.numpy())
            
datatype = "float"
n_output = "output_v"
n_input = "inputs"

dump(dedent(f"""
"""))

unroll = True

def matmul(A, n_b, n_x, s_input, s_output, n_activation, datatype, n_weights):
    if unroll:
        output = ""
        for j in range(s_output):
            for i in range(s_input):
                output += f"{n_b}[{j}] += {A[i][j]} * {n_x}[{i}];\n"

            output += f"{n_b}[{j}] = {n_activation}({n_b}[{j}]);\n"
        
        return output
    else:
        return dedent(f"""
            {datatype} {n_weights}[{s_input}][{s_output}] = {array2d_to_init_string(w)};
            for (int i = 0; i < {s_input}; i++) {{
                    for (int j = 0; j < {s_output}; j++) {{
                        {n_b}[j] += {n_weights}[i][j] * {n_x}[i];
                }}
            }}

            for (int i = 0; i < {s_output}; i++) {{
                {n_b}[i] = {n_activation}({n_b}[i]);
            }}
        """)

items = list(enumerate(zip(weights, biases, activation_func)))
for i, (w, b, a) in items:
    n_output_buf = f"output_layer{i}"
    n_weights = f"weights_layer{i}"

    input_size = w.shape[0]
    output_size = b.shape[0]
    
    if i == 0:
        initial_input_size = input_size
        prev_layer_name = n_input
        
    if i == len(items) - 1:
        n_output_buf = n_output
        last_layer_size = output_size
    
    dump(dedent(f"""
        {datatype} {n_output_buf}[{output_size}] = {array_to_init_string(b)};
        
        {matmul(w, n_output_buf, prev_layer_name, input_size, output_size, a, datatype, n_weights)}
    """))
    
    prev_layer_name = n_output_buf
        
workgroup_size = 1080
Path("shader.comp").write_text(dedent(f"""
#version 450

layout(std140, binding = 0) buffer readonly Inputs {{
    float[{workgroup_size}][{initial_input_size}] initial_input;
}};

layout(std140, binding = 1) buffer writeonly Output {{
    float[{workgroup_size}][{last_layer_size}] the_output;
}};

{datatype} linear({datatype} x) {{
    return x;
}}
{datatype} relu({datatype} x) {{
    return max(0, x);
}}

void main() {{
    float {n_input}[{initial_input_size}] = initial_input[gl_GlobalInvocationID.x];
    
    {indent(shader_code, '    ')}
    
    for (int i = 0; i < {last_layer_size}; i++) {{
        the_output[gl_GlobalInvocationID.x][i] = {n_output}[i];
    }}
}}
"""))


print_out(Path("shader.comp").read_text())



#version 450

layout(std140, binding = 0) buffer readonly Inputs {
    float[1080][113] initial_input;
};

layout(std140, binding = 1) buffer writeonly Output {
    float[1080][1] the_output;
};

float linear(float x) {
    return x;
}
float relu(float x) {
    return max(0, x);
}

void main() {
    float inputs[113] = initial_input[gl_GlobalInvocationID.x];




            float output_layer0[30] = {-1.2618301, 0.9302604, 0.2129137, 0.4299251, 0.52936184, 0.24...

            output_layer0[0] += 0.0052061728201806545 * inputs[0];
    output_layer0[0] += -0.01580808311700821 * inputs[1];
    output_layer0[0] += -0.00921562034636736 * inputs[2];
    output_layer0[0] += 0.019833246245980263 * inputs[3];
    output_layer0[0] += 0.025820495560765266 * inputs[4];
    output_layer0[0] += -0.056810829788446426 * inputs[5];
    output_layer0[0] += -0.01238161139190197 * inputs[6];
    output_layer0[0] += 0.045715514570474625 * inputs[7];
    output_layer0[0] += 0.05437552556395531 * inputs[8]

In [22]:
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import lava as lv
import numpy as np

# start session on first device
session = lv.Session(lv.devices()[0])

# compile glsl code
shader_path = "shader.comp"
shader = lv.Shader(session, lv.compile_glsl(shader_path))

# allocate buffer and take its definition from the shader (see glsl: float[HEIGHT][WIDTH][3] imageData)
buffer_inputs = lv.StagedBuffer.from_shader(session, shader, binding=0)
buffer_outputs = lv.StagedBuffer.from_shader(session, shader, binding=1)

# compute minimum amount of work groups
buffer_inputs[0] = np.stack(
    [np.arange(workgroup_size, dtype=np.float32)] * initial_input_size
).T
# buffer_inputs[0] = np.zeros_like(buffer_inputs[0])
# buffer_inputs[0][0][0] = 1

# bind buffer to binding 0 (see glsl: layout(std430, binding = 0) buffer buf)
stage = lv.Stage(shader, {0: buffer_inputs, 1: buffer_outputs})
stage.record(workgroup_size, 1, 1)
stage.run_and_wait()

# retrieve output as numpy array
data = buffer_outputs[0]

print(data)
#plt.imshow(np.around(255 * im).astype(np.uint8))

shader.comp
[[-2.7690003e+00]
 [ 8.4264755e+00]
 [ 1.4220929e+01]
 ...
 [ 7.9369087e+03]
 [ 7.9442808e+03]
 [ 7.9516514e+03]]


In [23]:
model.predict(buffer_inputs[0])
# first_layer_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
# print(first_layer_extractor.predict(buffer_inputs[0]))
# #print(np.where(np.array(biases[0]) > 0, biases[0], np.zeros_like(biases[0])))
# 
# 
# b = biases[0]
# w = weights[0]
# 
# output = np.copy(b)
# for i in range(113):
# #    for j in range(113):
#         for k in range(30):
#             output[k] += w[i,k] * buffer_inputs[0][0][i]
# print(output)
# print(np.matmul(w.T, buffer_inputs[0][0]) + b)

34/34 [==============================] - 0s 4ms/step


array([[-2.7690003e+00],
       [ 8.4264736e+00],
       [ 1.4220926e+01],
       ...,
       [ 7.9369082e+03],
       [ 7.9442793e+03],
       [ 7.9516499e+03]], dtype=float32)